In [146]:
import torch
import torchvision
import torchvision.datasets as datasets
from tqdm import tqdm
import argparse
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import torch.backends.cudnn as cudnn
import os
import sys
import logging
import logging.handlers
from PIL import Image

# Parser

In [240]:
parser = argparse.ArgumentParser(description='Domain adaptation')
parser.add_argument("--batch_size", type=int, default="100", help="batch size")
parser.add_argument("--learning_rate", type=float, default=1e-2, help="learning rate")
parser.add_argument("--momentum", type=float, default=0.5, help="momentum")
parser.add_argument("--gpu_num", type=int, default=0, help="gpu num")
parser.add_argument("--seed", type=int, default=123, help="munually set seed")
parser.add_argument("--save_path", type=str, default=r"C:\Users\Tianqin Li\Code\PGM-project\train_related", help="save path")
parser.add_argument("--subfolder", type=str, default=r'\baseline_st', help="subfolder name")
parser.add_argument("--wtarget", type=float, default=0.7, help="target weight")
parser.add_argument("--model_save_period", type=int, default=2, help="save period")


args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [--batch_size BATCH_SIZE]
                             [--learning_rate LEARNING_RATE]
                             [--momentum MOMENTUM] [--gpu_num GPU_NUM]
                             [--seed SEED] [--save_path SAVE_PATH]
                             [--subfolder SUBFOLDER] [--wtarget WTARGET]
                             [--model_save_period MODEL_SAVE_PERIOD]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\Tianqin Li\AppData\Roaming\jupyter\runtime\kernel-2c7b5274-f363-40d8-9c4b-ce24c168232c.json


SystemExit: 2

# local only

In [243]:
# local only
class local_args:
    def __init__(self, **entries):
        self.__dict__.update(entries)
        
args = local_args(**{
    'batch_size': 400,
    'learning_rate': 1e-2,
    'momentum': 0.5,
    'gpu_num': 0,
    'seed': 123,
    'save_path': r"C:\Users\Tianqin Li\Code\PGM-project\train_related",
    'epochs': 200,
    'subfolder': r'\dann_shffle',
    'wtarget': 0.7,
    'dann_weight': 1e-2,
    'model_save_period': 2,
})

In [244]:

device = torch.device('cuda:{}'.format(args.gpu_num) if torch.cuda.is_available() else 'cpu')
# seed
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)
np.random.seed(args.seed)
cudnn.deterministic = True
torch.backends.cudnn.deterministic = True



device = torch.device('cuda:{}'.format(args.gpu_num) if torch.cuda.is_available() else 'cpu')

 
model_sub_folder = args.subfolder + '\wtarget_%f_dann_weight_%f'%(args.wtarget, args.dann_weight)

if not os.path.exists(args.save_path+model_sub_folder):
    os.makedirs(args.save_path+model_sub_folder)

In [245]:

logger = logging.getLogger()
logger.setLevel(logging.INFO)

if os.path.isfile(args.save_path+model_sub_folder+ '\logfile.log'):
    os.remove(args.save_path+model_sub_folder+ '\logfile.log')
    
file_log_handler = logging.FileHandler(args.save_path+model_sub_folder+ '\logfile.log')
logger.addHandler(file_log_handler)

stdout_log_handler = logging.StreamHandler(sys.stdout)
logger.addHandler(stdout_log_handler)

attrs = vars(args)
for item in attrs.items():
    logger.info("%s: %s"%item)


batch_size: 400
batch_size: 400
batch_size: 400
batch_size: 400
batch_size: 400
batch_size: 400
learning_rate: 0.01
learning_rate: 0.01
learning_rate: 0.01
learning_rate: 0.01
learning_rate: 0.01
learning_rate: 0.01
momentum: 0.5
momentum: 0.5
momentum: 0.5
momentum: 0.5
momentum: 0.5
momentum: 0.5
gpu_num: 0
gpu_num: 0
gpu_num: 0
gpu_num: 0
gpu_num: 0
gpu_num: 0
seed: 123
seed: 123
seed: 123
seed: 123
seed: 123
seed: 123
save_path: C:\Users\Tianqin Li\Code\PGM-project\train_related
save_path: C:\Users\Tianqin Li\Code\PGM-project\train_related
save_path: C:\Users\Tianqin Li\Code\PGM-project\train_related
save_path: C:\Users\Tianqin Li\Code\PGM-project\train_related
save_path: C:\Users\Tianqin Li\Code\PGM-project\train_related
save_path: C:\Users\Tianqin Li\Code\PGM-project\train_related
epochs: 200
epochs: 200
epochs: 200
epochs: 200
epochs: 200
epochs: 200
subfolder: \dann_shffle
subfolder: \dann_shffle
subfolder: \dann_shffle
subfolder: \dann_shffle
subfolder: \dann_shffle
subfolder:

# Data loader

In [246]:
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))

In [247]:
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))

In [248]:
svhn_trainset = datasets.SVHN(root='./data', split='train', download=True, transform=torchvision.transforms.Compose([
                    torchvision.transforms.Resize((28, 28)),
                    torchvision.transforms.Grayscale(num_output_channels=1),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize([0.5], [0.5])]))

Using downloaded and verified file: ./data\train_32x32.mat


In [249]:
svhn_testset = datasets.SVHN(root='./data', split='test', download=True, transform=torchvision.transforms.Compose([
                    torchvision.transforms.Resize((28, 28)),
                    torchvision.transforms.Grayscale(num_output_channels=1),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize([0.5], [0.5])]))

Using downloaded and verified file: ./data\test_32x32.mat


In [250]:
# # mnist
# train_mnist_loader = DataLoader(mnist_trainset, batch_size=args.batch_size, shuffle=True)
# test_mnist_loader = DataLoader(mnist_testset, batch_size=args.batch_size, shuffle=True)
# examples = enumerate(test_mnist_loader)
# batch_idx, (example_data, example_targets) = next(examples)


# fig = plt.figure()
# for i in range(6):
#   plt.subplot(2,3,i+1)
#   plt.tight_layout()
#   plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
#   plt.title("Ground Truth: {}".format(example_targets[i]))
#   plt.xticks([])
#   plt.yticks([])

In [251]:
# # svhn
# train_svhn_loader = DataLoader(svhn_trainset, batch_size=args.batch_size, shuffle=True)
# test_svhn_loader = DataLoader(svhn_trainset, batch_size=args.batch_size, shuffle=True)
# examples = enumerate(train_svhn_loader)
# batch_idx, (example_data, example_targets) = next(examples)


# fig = plt.figure()
# for i in range(6):
#   plt.subplot(2,3,i+1)
#   plt.tight_layout()
#   plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
#   plt.title("Ground Truth: {}".format(example_targets[i]))
#   plt.xticks([])
#   plt.yticks([])

In [252]:
# reload data
train_mnist_loader = DataLoader(mnist_trainset, batch_size=args.batch_size, shuffle=True)
test_mnist_loader = DataLoader(mnist_testset, batch_size=args.batch_size, shuffle=True)
train_svhn_loader = DataLoader(svhn_trainset, batch_size=args.batch_size, shuffle=True)
test_svhn_loader = DataLoader(svhn_trainset, batch_size=args.batch_size, shuffle=True)

## Process data for cancat with source and target label

In [253]:
class ConcatDataset(Dataset):
    def __init__(self, x, y, mode='mnist'):
        self.x = x
        self.y = y
        self.len = self.x.shape[0]
        self.mode = mode
        if self.mode == 'mnist':
            self.transform = torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])
        elif self.mode == 'svhn':
            self.transform = torchvision.transforms.Compose([
                    torchvision.transforms.Resize((28, 28)),
                    torchvision.transforms.Grayscale(num_output_channels=1),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize([0.5], [0.5])])

    def __len__(self):
        return self.len
    
    def __getitem__(self, index):
        if self.mode == 'mnist':
            img = Image.fromarray(self.x[index].numpy(), mode='L')
            img = self.transform(img)
        elif self.mode == 'svhn':
            img = Image.fromarray(np.transpose(self.x[index], (1, 2, 0)))
            img = self.transform(img)
    
        return img, self.y[index]

In [254]:

concat_mnist_train = ConcatDataset(mnist_trainset.data, torch.randint(0,2,(mnist_trainset.data.shape[0],)), mode = 'mnist')
concat_svhn_train = ConcatDataset(svhn_trainset.data, torch.randint(0,2,(svhn_trainset.data.shape[0],)), mode = 'svhn')


adverial_dataset = torch.utils.data.ConcatDataset([concat_mnist_train, concat_svhn_train])
# [i[1] for i in [adverial_dataset[m] for m in torch.randint(0, len(adverial_dataset), (100,))]]
adverial_loader = DataLoader(adverial_dataset, batch_size=args.batch_size, shuffle=True)

# Model

In [255]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 30)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x

In [256]:
class FNN(nn.Module):
    def __init__(self, d_in, d_h1, d_h2, d_out, dp=0.2):
        super(FNN, self).__init__()
        self.fc1 = nn.Linear(d_in, d_h1)
        self.ln1 = nn.LayerNorm(d_h1)
        self.relu1 = nn.ReLU(inplace=True)
        self.dropout1 = nn.Dropout(dp)
        self.fc2 = nn.Linear(d_h1, d_h2)
        self.ln2 = nn.LayerNorm(d_h2)
        self.relu2 = nn.ReLU(inplace=True)
        self.dropout2 = nn.Dropout(dp)
        self.fc3 = nn.Linear(d_h2, d_out)

    def forward(self, x):
        x = self.fc1(x)
        x = self.ln1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.ln2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)

        return x

In [257]:
class Adversial_loss(nn.Module):
    def __init__(self):
        super(Adversial_loss, self).__init__()
    
    def forward(self):
        pass

In [258]:
def weights_init(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
    elif type(m) == nn.LayerNorm:
        torch.nn.init.normal_(m.weight, 1.0, 0.02)
        torch.nn.init.constant_(m.bias, 0)


In [259]:

device = torch.device('cuda:{}'.format(args.gpu_num) if torch.cuda.is_available() else 'cpu')
print(device)
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)
np.random.seed(args.seed)

encoder = Encoder().to(device)
CNet = FNN(d_in=30, d_h1=100, d_h2=100, d_out=10, dp=0.2).to(device)
DomainCNet = FNN(d_in=30, d_h1=100, d_h2=100, d_out=2, dp=0.2).to(device)




optimizerEncoder = optim.Adam(encoder.parameters(), lr=args.learning_rate)
optimizerCNet = optim.Adam(CNet.parameters(), lr=args.learning_rate)
optimizerDomainCNet = optim.Adam(DomainCNet.parameters(), lr=args.learning_rate)

criterion_classifier = nn.CrossEntropyLoss().to(device)
# criterion_adverisal = 

encoder.apply(weights_init)
CNet.apply(weights_init)
DomainCNet.apply(weights_init)

cuda:0


FNN(
  (fc1): Linear(in_features=30, out_features=100, bias=True)
  (ln1): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
  (relu1): ReLU(inplace=True)
  (dropout1): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (ln2): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
  (relu2): ReLU(inplace=True)
  (dropout2): Dropout(p=0.2, inplace=False)
  (fc3): Linear(in_features=100, out_features=2, bias=True)
)

# Train

In [260]:
target_acc_label_ = []
source_acc_ = []
source_test_acc_ = []
target_test_acc_ = []

logger.info('Started Training')


for epoch in range(args.epochs):
    # update classifier
    # on source domain mnist
    CNet.train()
    encoder.train()
    source_acc = 0.0
    num_datas = 0.0
    for batch_id, (source_x, source_y) in tqdm(enumerate(train_mnist_loader), total=len(train_mnist_loader)):
        optimizerCNet.zero_grad()
        optimizerEncoder.zero_grad()
        source_x = source_x.to(device).float()
        source_y = source_y.to(device)
        num_datas += source_x.size(0)
        source_x_embedding = encoder(source_x)
        pred = CNet(source_x_embedding)
        source_acc += (pred.argmax(-1) == source_y).sum().item()
        loss = criterion_classifier(pred, source_y)
        loss.backward()
        optimizerCNet.step()
        optimizerEncoder.step()
        
        
    source_acc = source_acc / num_datas
    source_acc_.append(source_acc)
    
    
    
    # on target domain svhn
    target_acc = 0.0
    num_datas = 0.0
    CNet.train()
    encoder.train()

    for batch_id, (target_x, target_y) in tqdm(enumerate(train_svhn_loader), total=len(train_svhn_loader)):
        optimizerCNet.zero_grad()
        optimizerEncoder.zero_grad()
        target_x = target_x.to(device).float()
        target_y = target_y.to(device)
        num_datas += target_x.size(0)
        target_x_embedding = encoder(target_x)
        pred = CNet(target_x_embedding)
        target_acc += (pred.argmax(-1) == target_y).sum().item()
        loss = criterion_classifier(pred, target_y)
        loss.backward()
        optimizerCNet.step()
        optimizerEncoder.step()
        
    
    target_acc = target_acc / num_datas
    target_acc_label_.append(target_acc)
    
    
    # DANN
    DomainCNet.train()
    encoder.train()
    num_datas = 0.0
    for batch_id, (adv_x, adv_y) in tqdm(enumerate(adverial_loader), total=len(adverial_loader)):
        optimizerCNet.zero_grad()
        optimizerEncoder.zero_grad()
        adv_x = adv_x.to(device).float()
        adv_y = adv_y.to(device)
        num_datas += adv_x.size(0)
        adv_x_embedding = encoder(adv_x)
        pred = DomainCNet(adv_x_embedding)
        # adv_acc += (pred.argmax(-1) == adv_y).sum().item()
        loss = args.dann_weight * criterion_classifier(pred, adv_y)
        loss.backward()
        optimizerEncoder.step()
        optimizerDomainCNet.step()
    
    
    
    
    
    # eval on source   
    source_test_acc = 0.0
    num_datas = 0.0
    CNet.eval()
    encoder.eval()
    
    for batch_id, (source_x, source_y) in enumerate(test_mnist_loader):
        optimizerCNet.zero_grad()
        optimizerEncoder.zero_grad()
        source_x = source_x.to(device).float()
        source_y = source_y.to(device)
        num_datas += source_x.size(0)
        source_x_embedding = encoder(source_x)
        pred = CNet(source_x_embedding)
        source_test_acc += (pred.argmax(-1) == source_y).sum().item()
        
    source_test_acc = source_test_acc / num_datas
    source_test_acc_.append(source_test_acc)
    
    # eval on target 
    num_datas = 0.0
    target_test_acc = 0.0
    for batch_id, (target_x, target_y) in enumerate(test_svhn_loader):
        optimizerCNet.zero_grad()
        optimizerEncoder.zero_grad()
        target_x = target_x.to(device).float()
        target_y = target_y.to(device)
        num_datas += target_x.size(0)
        target_x_embedding = encoder(target_x)
        pred = CNet(target_x_embedding)
        target_test_acc += (pred.argmax(-1) == target_y).sum().item()
    
    target_test_acc = target_test_acc / num_datas
    target_test_acc_.append(source_test_acc)
    
    if epoch % args.model_save_period == 0:
        torch.save(DomainCNet.state_dict(), args.save_path+model_sub_folder+ '/DomainCNet_%i.t7'%(epoch+1))
        torch.save(encoder.state_dict(), args.save_path+model_sub_folder+ '/encoder_%i.t7'%(epoch+1))
        torch.save(CNet.state_dict(), args.save_path+model_sub_folder+ '/CNet_%i.t7'%(epoch+1))

    
    logger.info('Epochs %i: source train acc: %f; source test acc: %f; target train acc: %f; target test acc: %f'%(epoch+1, source_acc, source_test_acc, target_acc, target_test_acc))
    

Started Training
Started Training
Started Training
Started Training
Started Training
Started Training


100%|██████████| 334/334 [00:20<00:00, 16.54it/s]


Epochs 1: source train acc: 0.606250; source test acc: 0.555700; target train acc: 0.361727; target test acc: 0.506109
Epochs 1: source train acc: 0.606250; source test acc: 0.555700; target train acc: 0.361727; target test acc: 0.506109
Epochs 1: source train acc: 0.606250; source test acc: 0.555700; target train acc: 0.361727; target test acc: 0.506109
Epochs 1: source train acc: 0.606250; source test acc: 0.555700; target train acc: 0.361727; target test acc: 0.506109
Epochs 1: source train acc: 0.606250; source test acc: 0.555700; target train acc: 0.361727; target test acc: 0.506109
Epochs 1: source train acc: 0.606250; source test acc: 0.555700; target train acc: 0.361727; target test acc: 0.506109


100%|██████████| 334/334 [00:20<00:00, 16.67it/s]


Epochs 2: source train acc: 0.842917; source test acc: 0.687600; target train acc: 0.547211; target test acc: 0.639038
Epochs 2: source train acc: 0.842917; source test acc: 0.687600; target train acc: 0.547211; target test acc: 0.639038
Epochs 2: source train acc: 0.842917; source test acc: 0.687600; target train acc: 0.547211; target test acc: 0.639038
Epochs 2: source train acc: 0.842917; source test acc: 0.687600; target train acc: 0.547211; target test acc: 0.639038
Epochs 2: source train acc: 0.842917; source test acc: 0.687600; target train acc: 0.547211; target test acc: 0.639038
Epochs 2: source train acc: 0.842917; source test acc: 0.687600; target train acc: 0.547211; target test acc: 0.639038


...

Epochs 198: source train acc: 0.940950; source test acc: 0.974100; target train acc: 0.777386; target test acc: 0.810557
Epochs 198: source train acc: 0.940950; source test acc: 0.974100; target train acc: 0.777386; target test acc: 0.810557
Epochs 198: source train acc: 0.940950; source test acc: 0.974100; target train acc: 0.777386; target test acc: 0.810557
Epochs 198: source train acc: 0.940950; source test acc: 0.974100; target train acc: 0.777386; target test acc: 0.810557
Epochs 198: source train acc: 0.940950; source test acc: 0.974100; target train acc: 0.777386; target test acc: 0.810557
Epochs 198: source train acc: 0.940950; source test acc: 0.974100; target train acc: 0.777386; target test acc: 0.810557


100%|██████████| 334/334 [00:20<00:00, 16.62it/s]


Epochs 199: source train acc: 0.941100; source test acc: 0.975200; target train acc: 0.777837; target test acc: 0.827457
Epochs 199: source train acc: 0.941100; source test acc: 0.975200; target train acc: 0.777837; target test acc: 0.827457
Epochs 199: source train acc: 0.941100; source test acc: 0.975200; target train acc: 0.777837; target test acc: 0.827457
Epochs 199: source train acc: 0.941100; source test acc: 0.975200; target train acc: 0.777837; target test acc: 0.827457
Epochs 199: source train acc: 0.941100; source test acc: 0.975200; target train acc: 0.777837; target test acc: 0.827457
Epochs 199: source train acc: 0.941100; source test acc: 0.975200; target train acc: 0.777837; target test acc: 0.827457


100%|██████████| 334/334 [00:20<00:00, 16.64it/s]


Epochs 200: source train acc: 0.940050; source test acc: 0.969500; target train acc: 0.777878; target test acc: 0.841503
Epochs 200: source train acc: 0.940050; source test acc: 0.969500; target train acc: 0.777878; target test acc: 0.841503
Epochs 200: source train acc: 0.940050; source test acc: 0.969500; target train acc: 0.777878; target test acc: 0.841503
Epochs 200: source train acc: 0.940050; source test acc: 0.969500; target train acc: 0.777878; target test acc: 0.841503
Epochs 200: source train acc: 0.940050; source test acc: 0.969500; target train acc: 0.777878; target test acc: 0.841503
Epochs 200: source train acc: 0.940050; source test acc: 0.969500; target train acc: 0.777878; target test acc: 0.841503
